In [7]:
# pip install openai sqlite-vec
from openai import OpenAI
import sqlite3
import sqlite_vec
import struct
from typing import List
import requests
import re

### Init DB

In [9]:
# init db
with sqlite3.connect('test_chess_vec.db') as conn:
    curr = conn.cursor()
    conn.enable_load_extension(True)
    sqlite_vec.load(conn)
    conn.enable_load_extension(False)
    
    # basic schema
    curr.execute(
        '''
        CREATE TABLE IF NOT EXISTS users (
            username TEXT UNIQUE, 
            last_updated TEXT
            )
        '''
    )
    curr.execute(
        '''
        CREATE TABLE IF NOT EXISTS games (
            url TEXT UNIQUE,
            pgn TEXT,
            time_control TEXT,
            end_time TEXT,
            rated TEXT,
            accuracies_white REAL,
            accuracies_black REAL,
            tcn TEXT,
            uuid TEXT,
            initial_setup TEXT,
            fen TEXT,
            time_class TEXT,
            rules TEXT,
            white_rating INTEGER,
            white_result TEXT,
            white_username TEXT,
            white_uuid TEXT,
            black_rating INTEGER,
            black_result TEXT,
            black_username TEXT,
            black_uuid TEXT,
            eco TEXT 
        )
        '''
    )

    curr.execute(
    """
        CREATE VIRTUAL TABLE pgn_embeddings USING vec0(
            url TEXT PRIMARY KEY UNIQUE,
            pgn_embedding FLOAT[1536]
        );
    """
    )

    conn.commit()

### Insert Test Data

In [11]:
from utils import add_or_update_user, fetch_user_archives

username = 'notjoemartinez'

add_or_update_user(username)
fetch_user_archives(username)


Updating notjoemartinez
Inserting games into database...


### Add embeddings to pgn_embeddings

In [ ]:
def serialize(vector: List[float]) -> bytes:
    """serializes a list of floats into a compact "raw bytes" format"""
    return struct.pack("%sf" % len(vector), *vector)

def get_embedding(text, model="text-embedding-3-small"):

    client = OpenAI()
    text_embedding = client.embeddings.create(input=[text], model=model).data[0].embedding

    return text_embedding


username = 'notjoemartinez'

with sqlite3.connect('test_chess_vec.db') as conn:
    curr = conn.cursor()

    curr.execute(
        """
        SELECT url, pgn FROM games
        WHERE white_username = ? OR black_username = ?
        """, (username, username)
    )
    
    games = curr.fetchall()

    for url, pgn in games:
        print(url)
        pgn_embedding = get_embedding(pgn)
        curr.execute(
                "INSERT OR IGNORE INTO pgn_embeddings(url, pgn_embedding) VALUES(?, ?)",
                [url, serialize(pgn_embedding)],
            )
